In [1]:
from langchain_google_genai import GoogleGenerativeAI

api_key = '' # enter your google api key here

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
print(
    llm.invoke(
        "introduce yourself"
    )
)

Greetings! My name is Gemini, and I'm an AI chatbot assistant. I'm here to assist you with a wide range of inquiries and discussions to the best of my abilities. Whether you have questions, need information, or simply want to engage in conversation, feel free to ask. I'll do my best to provide helpful and informative responses. While I may not possess the full spectrum of human knowledge and experience, I'm continuously learning and expanding my capabilities. Let's explore and discover together!


## Now let's load data from Codebasics FAQ csv file

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='QA-System-for-E_Learning/codebasics_faqs.csv', source_column='prompt', encoding='latin')

data = loader.load()
data

[Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
 Document(page_content='prompt: Why should I trust Codebasics?\nresponse: Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.', metadata={'source': 'Why should I trust Codebasics?', 'row': 1}),
 Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you ne

## Hugging Face Embeddings

In [3]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# Inititlaize instructor embeddings using teh Hugging Face model
embeddings = HuggingFaceInstructEmbeddings()

# try an example text to embed
text = "What is your refund policy?"
e = embeddings.embed_query(text)

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
len(e)

768

## Vector store using FAISS

In [5]:
from langchain.vectorstores import FAISS

# create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents= data, embedding=embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [6]:
rdocs = retriever.invoke('how about job placement support?')
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

- As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

## Create RetrievalQA chain along with prompt template

In [11]:
# import PromptTemplate
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=['context' , 'question']
)

chain_type_kwargs = {'prompt': PROMPT}


# import RetrivalQA
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type='stuff',
                                   retriever=retriever,
                                   input_key='query',
                                   return_source_documents = True,
                                   chain_type_kwargs=chain_type_kwargs)

## We are all set 👍🏼 Let's ask some questions now

In [14]:
chain.invoke('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.\nHowever, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done

In [15]:
chain.invoke("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "Yes, we provide internships. No, we don't offer EMI payments.",
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: How can I contact the instructors for any doubts/support?\nresponse: We have created every lecture with a motive to explain everything in an easy-to-und